## 개념 정리
### 용어 정리
- 검증세트: 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트셋을 사용하지 않기 위해 훈련셋에서 분리한 데이터
- 교차검증: 훈련셋을 여러 개의 셋으로 나누고, 돌아가며 검증셋과 학습셋 역할을 하며 모델을 학습. 모든 폴드의 검증 점수를 얻어 평균점수로 판단하는 방법
- 그리드서치: 하이퍼파라미터 탐색을 자동화 해주는 도구. 가장 좋은 점수의 파라미터를 고르고, 자동으로 모델을 훈련함
- 랜덤서치: 랜덤한 값을 파라미터로 주고, 그 안에서 최적의 파라미터 값을 찾는 과정.
- 하이퍼파라미터 과정
    - 탐색할 매개변수를 지정
    - 훈련셋에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 이것을 그리드 서치 객체에 저장함
    - 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련함. 이 모델도 그리드 서치 객체에 저장함
### 핵심 패키지와 함수(sklearn)
- cross_validate()
    - 교차검증을 수행하는 함수
    - scoring 매개변수를 통해, 평가지표를 선택가능 ('accuracy' or 'r2')
    - cv 매개변수를 통해, 교차 검증 폴드 수나 스플리터 객체를 지정가능
    - n_jobs 매개변수를 통해, 사용할 CPU 코어수를 설정. -1은 모든 코어를 사용함.
    - return_trace_score 매개변수를 True로 선택하면 훈련 셋의 점수도 반환함.
- GridSearchCV
    - 교차 검증으로 하이퍼파라미터의 탐색을 수행함.
    - 최상의 모델을 찾은 후 훈련셋 전체를 사용해 최종 모델을 훈련함.
    - 그리드 서치를 수행할 모델과 파라미터를 전달함
    - 기타 매개변수는 cross_validate()와 동일
- RandomizedSearchCV
    - 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행함
    - 최상의 모델을 찾은 후 훈련셋 전체를 사용해 최종 모델을 훈련함.
    - 그리드 서치를 수행할 모델과 파라미터(확률분포)를 전달함
    - 기타 매개변수는 cross_validate()와 동일
### 핵심 패키지와 함수(scipy)
- randint(): 정수 난수 생성
- uniform(): 실수 난수 생성

In [2]:
'''데이터 불러오기'''
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine[['class']].to_numpy()

In [3]:
'''데이터 나누기'''
# 훈련셋과 테스트셋 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 훈련셋과 검증셋 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [4]:
'''모델 훈련 하기'''
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=42)
DT.fit(sub_input, sub_target)
print(DT.score(sub_input, sub_target))
print(DT.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
'''교차검증 수행하기'''

# 분할기를 사용하지 않은 교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(DT, train_input, train_target)
print(scores)

import numpy as np
print(np.mean(scores['test_score'])) # Fold 점수의 평균을 구함

# 분할기를 사용한 교차검증
# 회귀문제에서는 k-fold를, 분류문제에서는 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(DT, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score'])) # Fold 점수의 평균을 구함

# 훈련셋을 섞은 후, 교차검증 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(DT, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

{'fit_time': array([0.00498629, 0.00498676, 0.00598431, 0.00499058, 0.00498319]), 'score_time': array([0.        , 0.0009973 , 0.0009973 , 0.00099754, 0.00099635]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719


In [6]:
'''하이퍼파라미터 튜닝'''
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # min_impurtiy_decrease에 해당하는 파라미터를 5개 설정
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # 앞서 설정한 파라미터를 모델에 대입 # cv 기본값은 5이므로, 5-fold 검증 시행함. 즉, 5*5=25번의 모델학습을 수행하도록 함. # n_jobs = -1은 모든 코어를 사용함
gs.fit(train_input, train_target) # 모델 학습
DT = gs.best_estimator_

# 결과 출력
print(DT.score(train_input, train_target))
print(gs.best_params_) # 최적의 파라미터 출력
print(gs.cv_results_['mean_test_score']) # 파라미터별 교차검증의 평균점수 출력

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [7]:
'''조금 더 복잡한 파라미터 이용한 하이퍼파라미터 튜닝'''
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split': range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# 결과 출력
print(gs.best_params_) # 파라미터별 최적의 값 출력
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수 출력

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [9]:
'''랜덤 서치'''
# 난수 생성
from scipy.stats import uniform, randint

# randint(): 정수 난수 생성
# uniform(): 실수 난수 생성

params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)
          }

# 하이퍼파라미터 찾기
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_ # 최적의 모델은 best_estimator에 저장되어 있음
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
